In [ ]:
#
# Using Uniform Sample Distribution To Better Distribute the Nodes
#
import polars   as pl
import networkx as nx
import rtsvg
rt = rtsvg.RACETrack()

# Parse the file format into a dataframe
edges_filename, layout_filename  = '../../data/stanford/facebook/414.edges', '../../data/stanford/facebook/414.rt_layout.parquet'
_lu_ = {'fm':[], 'to':[]}
for _edge_ in open(edges_filename, 'rt').read().split('\n'):
    if _edge_ == '': continue
    _split_     = _edge_.split()
    _fm_, _to_  = int(_split_[0]), int(_split_[1])
    _lu_['fm'].append(_fm_), _lu_['to'].append(_to_)
df  = pl.DataFrame(_lu_)

# Relationship & networkx graph w/ a spring layout
_relates_ = [('fm','to')]
g   = rt.createNetworkXGraph(df, _relates_)
pos = nx.spring_layout(g)

# Expands nodes using Uniformm Sample Distribution algorithm / better filling the space on the screen
_nodes_                   = list(pos.keys())
_xs_, _ys_, _ws_, _ns_ = [], [], [], []
for i in range(len(_nodes_)):
    _xs_   .append(pos[_nodes_[i]][0])
    _ys_   .append(pos[_nodes_[i]][1])
    _ws_   .append(g.degree(_nodes_[i]))
    _ns_   .append(_nodes_[i])
_df_         = pl.DataFrame({'x':_xs_, 'y':_ys_, 'w':_ws_, 'node':_ns_})
_df_results_ = rt.uniformSampleDistributionInScatterplotsViaSectorBasedTransformation(_df_, 'x', 'y', wgt_field='w', iterations=128)
_pos_ = {}
for i in range(len(_df_results_)): _pos_[_df_results_['node'][i]] = (_df_results_['x'][i], _df_results_['y'][i])

# Provides colors based on community
_node_colors_ = {}
community_i   = 0
for _community_ in nx.community.louvain_communities(g):
    community_i += 1
    for _node_ in _community_: _node_colors_[_node_] = rt.co_mgr.getColor(community_i)
_ln_params_ = {'relationships':_relates_, 'pos':_pos_, 'color_by':None, 'node_color':_node_colors_, 'link_shape':'curve', 'link_size':'small', 'draw_labels':False}

# Interactive Graph Layout
_igl_ = rt.interactiveGraphLayout(df, _ln_params_, w=600, h=600)
_igl_